In [1]:
import re
import html
import subprocess
import os

In [106]:
# run yt-dlp to get the 2 language subs
result = subprocess.run(["yt-dlp", "--write-sub", "--sub-lang", "en" 
                            "--skip-download", "https://www.youtube.com/watch?v=3ymwOvzhwHs"])

In [111]:
url = "https://www.youtube.com/watch?v=JByz3mCM0KM"
os.system(f"yt-dlp --write-sub --sub-lang en,ko --skip-download {url} -o a > /dev/null 2>&1")

[youtube] Extracting URL: https://www.youtube.com/watch?v=JByz3mCM0KM
[youtube] JByz3mCM0KM: Downloading webpage
[youtube] JByz3mCM0KM: Downloading android player API JSON
[youtube] JByz3mCM0KM: Downloading android player API JSON
[youtube] JByz3mCM0KM: Downloading android player API JSON
[youtube] JByz3mCM0KM: Downloading android player API JSON


[info] JByz3mCM0KM: Downloading subtitles: en, ko
[info] JByz3mCM0KM: Downloading 1 format(s): 248+251
[info] Writing video subtitles to: a.en.vtt
[download] Destination: a.en.vtt
[download] 100% of   23.50KiB in 00:00:00 at 638.82KiB/s
[info] Writing video subtitles to: a.ko.vtt
[download] Destination: a.ko.vtt
[download] 100% of   27.94KiB in 00:00:00 at 346.36KiB/s


0

In [2]:
with open("a.en.vtt") as f:
    en = html.unescape(f.read())
with open("a.ko.vtt") as f:
    ko = html.unescape(f.read())



In [3]:
def to_seconds(s):
    hours, minutes, seconds, milliseconds = map(int, s.replace(".", ":").split(":"))
    total_seconds = hours * 3600 + minutes * 60 + seconds + milliseconds / 1000
    return total_seconds

In [4]:
class SubtitleEntry:
    def __init__(self, start, end, text):
        self.start_txt = start
        self.start = to_seconds(start)
        self.end = to_seconds(end)
        self.text = text
    def __str__(self):
        return self.text

In [5]:
def make_sub_list(text):
    pattern = r"(\d{2}:\d{2}:\d{2}\.\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2}\.\d{3})\n(.*?)\n"
    matches = re.finditer(pattern, text, re.DOTALL)
    timestamp_list = []
    for match in matches:
        start_timestamp = match.group(1)
        end_timestamp = match.group(2)
        following_line = match.group(3)
        timestamp_list.append([start_timestamp, end_timestamp, following_line])
    subs = [SubtitleEntry(*x) for x in timestamp_list]
    return subs

In [10]:
en_subs = make_sub_list(en)
ko_subs = make_sub_list(ko)
res = ''

In [11]:
time_loc = 0

In [12]:
# for each ko sub, get all en subs that start before the ko sub ends
en_index = 0
ko_index = 0
while en_index < len(en_subs) and ko_index < len(ko_subs):
    print(time_loc)
    print(ko_subs[ko_index].start)
    if ko_subs[ko_index].start > time_loc:
        res += "# --> " + ko_subs[ko_index].start_txt + '\n'
        time_loc += 150
    res += ko_subs[ko_index].text + '\n'
    while en_index < len(en_subs) and ko_subs[ko_index].end > en_subs[en_index].start:
        res += en_subs[en_index].text + '\n'
        en_index +=1
    res += '\n'        
    ko_index += 1
     

0
5.21
150
9.21
150
12.81
150
15.62
150
19.05
150
21.55
150
28.76
150
35.3
150
40.21
150
47.21
150
54.82
150
61.09
150
69.03
150
73.67
150
81.15
150
83.65
150
90.39
150
101.2
150
109.44
150
112.38
150
117.18
150
122.06
150
129.43
150
134.73
150
140.57
150
145.41
150
149.58
150
153.92
300
157.46
300
158.93
300
162.83
300
166.77
300
170.67
300
171.74
300
173.07
300
177.41
300
183.38
300
187.45
300
190.69
300
194.09
300
197.4
300
202.77
300
209.54
300
214.31
300
219.09
300
220.92
300
225.43
300
231.33
300
235.2
300
238.2
300
246.55
300
249.65
300
255.22
300
263.03
300
265.76
300
266.93
300
275.54
300
282.88
300
290.16
300
309.31
450
316.92
450
321.45
450
323.76
450
325.09
450
332.26
450
336.4
450
340.74
450
348.25
450
353.22
450
365.83
450
374.07
450
382.38
450
387.19
450
396.26
450
403.2
450
408.37
450
411.11
450
413.31
450
414.78
450
419.55
450
421.62
450
427.16
450
431.6
450
435.57
450
438.97
450
446.21
450
450.38
600
453.52
600
457.92
600
462.73
600
471.94
600
478.01
600
481.61
600
48

In [13]:
# print(res)

In [14]:
with open("par.txt", "w") as file:
    file.write(res)

In [17]:
os.system('pandoc par.txt -o output.epub --metadata title="Your Title" --metadata author="Didi" --wrap=none')

0